# Modeling & Inference

### Importing Libraries

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from feature_engine.selection import RecursiveFeatureElimination
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder
from feature_engine.encoding import StringSimilarityEncoder
from feature_engine.encoding import OrdinalEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.preprocessing import StandardScaler
from feature_engine.encoding import OneHotEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import average_precision_score
from scipy.stats import ks_2samp
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

### Reading the data

In [2]:
# Read CSVs
real_world_df = pd.read_csv('./data/real-world-data/healthcare-dataset-stroke-data.csv', index_col=0)
train_synthetic_df = pd.read_csv("./data/synthetic-data/train.csv", index_col=0)
test_df = pd.read_csv('./data/synthetic-data/test.csv', index_col=0)

In [3]:
df = pd.concat([train_synthetic_df, real_world_df], ignore_index=True, axis=0)
df.reset_index(drop=True, inplace=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [4]:
# Features
s_categorical = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
binary = ["hypertension", "heart_disease"] # Basically, categorical values with only 2 values. 1 or 0
continous_numerical = ["age", "avg_glucose_level", "bmi"]

### Using a Decision Tree to predict the missing BMI

In [5]:
DT_bmi_pipe = Pipeline( steps=[ 
                               ("scale",StandardScaler()),
                               ("lr",DecisionTreeRegressor(random_state=42))
                              ])
dt_df = df.copy()
X = dt_df[["age","gender","bmi"]].copy()
X.gender = X.gender.replace({"Male":0,"Female":1,"Other":-1}).astype(np.uint8)

missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop("bmi")
DT_bmi_pipe.fit(X,Y)
predicted_bmi = pd.Series(DT_bmi_pipe.predict(missing[["age","gender"]]),index=missing.index)
dt_df.loc[missing.index,"bmi"] = predicted_bmi

print("Missing values after decision tree regressor: ",sum(dt_df.isnull().sum()))

Missing values after decision tree regressor:  0


### Using a Simple Imputer

In [6]:
mean_imputer = SimpleImputer(strategy="mean")
imputed_df = df.copy()
imputed_df["bmi"] = mean_imputer.fit_transform(imputed_df["bmi"].values.reshape(-1,1))
print("Missing values after imputing: ",sum(imputed_df.isnull().sum()))

Missing values after imputing:  0


### Baseline

In [7]:
@ignore_warnings(category=ConvergenceWarning)
def baseline(train_df, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler):
    # cast categorical features as categorical type
    train_df[s_categorical] = (train_df[s_categorical].astype("category"))
    for fold, (train_indx, val_indx) in enumerate(cv.split(train_df, target)):
        X_train, Y_train = train_df.iloc[train_indx], target.iloc[train_indx]
        X_val, Y_val = train_df.iloc[val_indx], target.iloc[val_indx]
        
        X_train = X_train.copy()
        X_val = X_val.copy()
        


        X_train = encoder.fit_transform(X_train)
        X_train = scaler.fit_transform(X_train)
        
        X_val = encoder.transform(X_val)
        X_val = scaler.transform(X_val)

        print("_"*50)
        
        model.fit(X_train, Y_train)
        
        oof_preds.iloc[val_indx] = model.predict_proba(X_val)[:, 1]
        train_auc.append(roc_auc_score(Y_train, model.predict_proba(X_train)[:, 1]))
        val_auc.append(roc_auc_score(Y_val, model.predict_proba(X_val)[:, 1]))
        pipelines.append([encoder, scaler, model])

        print(f"Val AUC: {val_auc[-1]}")
    return pipelines, train_auc, val_auc, oof_preds

Pipeline with Logistic Regression + OneHot Encoder

In [8]:
cv = StratifiedKFold(shuffle=True, random_state=42)
features = s_categorical + continous_numerical
model = LogisticRegression()
encoder = OneHotEncoder(drop_last=True, variables=s_categorical)
scaler = SklearnTransformerWrapper(StandardScaler(), variables=continous_numerical)

In [9]:
train_auc = []
val_auc = []
pipelines = []
oof_preds = pd.Series(0, index=train_synthetic_df.index)

train_df = train_synthetic_df[features].copy()
target = train_synthetic_df["stroke"]

_, train_auc, val_auc, oof_preds_synth = baseline(train_df, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler)
print()
print(f"Synthetic Train AUC: {np.mean(train_auc)}")
print(f"Synthetic Val AUC: {np.mean(val_auc)}")
print(f"Synthetic OOF AUC score: {roc_auc_score(target, oof_preds_synth)}")

__________________________________________________
Val AUC: 0.882504529352911
__________________________________________________
Val AUC: 0.8824828966225899
__________________________________________________
Val AUC: 0.8923830840163384
__________________________________________________
Val AUC: 0.8760177983886983
__________________________________________________
Val AUC: 0.8797080750045986

Synthetic Train AUC: 0.8849088476136592
Synthetic Val AUC: 0.8826192766770273
Synthetic OOF AUC score: 0.8824682638419171


In [10]:
train_auc = []
val_auc = []
pipelines = []
oof_preds = pd.Series(0, index=imputed_df.index)

imputed_df_train = imputed_df[features].copy()
target = imputed_df["stroke"]

_, train_auc, val_auc, oof_preds_imputed = baseline(imputed_df_train, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler)
print()
print(f"Imputed with mean Train AUC: {np.mean(train_auc)}")
print(f"Imputed with mean Val AUC: {np.mean(val_auc)}")
print(f"Imputed with mean OOF AUC score: {roc_auc_score(target, oof_preds_imputed)}")

__________________________________________________
Val AUC: 0.8719053214510086
__________________________________________________
Val AUC: 0.8716042883402749
__________________________________________________
Val AUC: 0.8677119447611514
__________________________________________________
Val AUC: 0.881850607930433
__________________________________________________
Val AUC: 0.8670329446539125

Imputed with mean Train AUC: 0.8736126228638372
Imputed with mean Val AUC: 0.872021021427356
Imputed with mean OOF AUC score: 0.8719507131706968


In [11]:
train_auc = []
val_auc = []
pipelines = []
oof_preds = pd.Series(0, index=dt_df.index)

dt_df_train = dt_df[features].copy()
target = dt_df["stroke"]

_, train_auc, val_auc, oof_preds_dt = baseline(dt_df_train, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler)
print()
print(f"Desicion Tree Train AUC: {np.mean(train_auc)}")
print(f"Desicion Tree Val AUC: {np.mean(val_auc)}")
print(f"Desicion Tree OOF AUC score: {roc_auc_score(target, oof_preds_dt)}")

__________________________________________________
Val AUC: 0.8718835074574773
__________________________________________________
Val AUC: 0.8715606603532123
__________________________________________________
Val AUC: 0.8678384659236336
__________________________________________________
Val AUC: 0.8819489645077399
__________________________________________________
Val AUC: 0.8672060466415304

Desicion Tree Train AUC: 0.8736964244001928
Desicion Tree Val AUC: 0.8720875289767186
Desicion Tree OOF AUC score: 0.8720377046952121


We see that synthetic data fits the best to our pipeline. Let's try other encoders.

In [12]:
encoder = CountFrequencyEncoder(variables=s_categorical)
train_auc = []
val_auc = []
pipelines = []
oof_preds = pd.Series(0, index=train_synthetic_df.index)
target = train_synthetic_df["stroke"]
pipelines, train_auc, val_auc, oof_preds_f = baseline(train_df, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler)
print()
print(f"Synthetic CountFrequencyEncoder Train AUC: {np.mean(train_auc)}")
print(f"Synthetic CountFrequencyEncoder Val AUC: {np.mean(val_auc)}")
print(f"Synthetic CountFrequencyEncoder OOF AUC: {roc_auc_score(target, oof_preds_f)}")

__________________________________________________
Val AUC: 0.8828614694032071
__________________________________________________
Val AUC: 0.8822422324977691
__________________________________________________
Val AUC: 0.8924340745750339
__________________________________________________
Val AUC: 0.8686778416501618
__________________________________________________
Val AUC: 0.875850726566473

Synthetic CountFrequencyEncoder Train AUC: 0.8817984049890443
Synthetic CountFrequencyEncoder Val AUC: 0.8804132689385291
Synthetic CountFrequencyEncoder OOF AUC: 0.8802621112460831


In [13]:
encoder = StringSimilarityEncoder(variables=s_categorical)
train_auc = []
val_auc = []
pipelines = []
oof_preds = pd.Series(0, index=train_synthetic_df.index)
target = train_synthetic_df["stroke"]
pipelines, train_auc, val_auc, oof_preds_ss = baseline(train_df, cv, oof_preds, target, train_auc, val_auc, pipelines, model, encoder, scaler)
print()
print(f"Synthetic StringSimilarityEncoder Train AUC: {np.mean(train_auc)}")
print(f"StringSimilarityEncoder Val AUC: {np.mean(val_auc)}")
print(f"StringSimilarityEncoder OOF AUC score: {roc_auc_score(target, oof_preds_ss)}")

__________________________________________________
Val AUC: 0.8825829480003246
__________________________________________________
Val AUC: 0.8824801925312998
__________________________________________________
Val AUC: 0.8924743302792673
__________________________________________________
Val AUC: 0.8759990123933894
__________________________________________________
Val AUC: 0.8797432401726881

Synthetic StringSimilarityEncoder Train AUC: 0.8849180160213044
StringSimilarityEncoder Val AUC: 0.8826559446753939
StringSimilarityEncoder OOF AUC score: 0.8825379306834338


It seems that StringSimilarityEncoder gives the highest score but its method doesn't make much sense to us. Our best choice for encoders is OneHotEncoder.   
We will try a submission with StringSimilarityEncoder as well.

>The Kolmogorov–Smirnov test is a nonparametric goodness-of-fit test and is used to determine wether two distributions differ, or whether an underlying probability distribution differes from a hypothesized distribution. It is used when we have two samples coming from two populations that can be different.  

>If the p-value is lower than our threshold of 0.05, so we reject the null hypothesis in favor of the default “two-sided” alternative: the data were not drawn from the same distribution.

In [14]:
ks_2samp(oof_preds_synth[target==0], oof_preds_synth[target==1])

KstestResult(statistic=0.6091470190356556, pvalue=2.4700771372158504e-216, statistic_location=0.0410081412417565, statistic_sign=1)

### Trying RecursiveFeatureElimination

In [15]:
# initialize linear regresion estimator
X = train_synthetic_df.drop("stroke", axis=1)
y = train_synthetic_df["stroke"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
linear_model = LinearRegression()

# initialize feature selector
tr = RecursiveFeatureElimination(estimator=linear_model, scoring="r2", cv=3)
Xt = tr.fit_transform(X_train, y_train)
print(tr.initial_model_performance_)
print(tr.performance_drifts_)
print(tr.features_to_drop_)
del X, y, X_train, X_test, y_train, y_test, linear_model, tr, Xt

0.0900333374972037
{'avg_glucose_level': 0.0114814154900168, 'bmi': 0.002188074390195749, 'age': 0.04421995300609483, 'heart_disease': 0.000671441535687517, 'hypertension': 0.006686764700367692}
['hypertension', 'heart_disease', 'bmi']


### Feature Engineering

In [16]:
def generate_features(df):
    df['age/bmi'] = df.age / df.bmi
    df['age*bmi'] = df.age * df.bmi
    df['bmi/prime'] = df.bmi / 25
    df['obesity'] = df.avg_glucose_level * df.bmi / 1000
    df['blood_heart']= df.hypertension*df.heart_disease
    return df

In [17]:
categorical = s_categorical + binary
features = categorical + continous_numerical + ['age/bmi', 'age*bmi', 'bmi/prime', 'obesity', 'blood_heart']
rfe_features =[
    'age', 'avg_glucose_level', 'bmi', 'ever_married_Yes', 'gender_Male',
    'hypertension_0', 'smoking_status_Unknown', 'smoking_status_formerly smoked',
    'smoking_status_never smoked', 'work_type_Self-employed', 'work_type_children'
]

fill_unknown = 'never smoked'

In [18]:
real_world_df = pd.read_csv(
    './data/real-world-data/healthcare-dataset-stroke-data.csv',
    index_col=0
)

train_df = pd.read_csv('./data/synthetic-data/train.csv', index_col=0)
test_df = pd.read_csv('./data/synthetic-data/test.csv', index_col=0)



In [19]:
def XGBoost_model(synthetic_train_df, real_world_df, train_df, test_df_c, real_world_target, encoder):
    # cast categorical features as categorical type
    synthetic_train_df[categorical] = (synthetic_train_df[categorical].astype('category'))
    real_world_df[categorical] = (real_world_df[categorical].astype('category'))


    target = train_df['stroke']
    # Grab Target
    oof_preds = pd.Series(0, index=train_df.index, name='stroke')
    test_preds = pd.Series(0, index=test_df.index, name='stroke')

    train_auc, val_auc = [], []
    pipelines = []

    for fold, (train_indx, val_indx) in enumerate(cv.split(train_df, target)):
        pipeline = []
        X_train, y_train = synthetic_train_df.iloc[train_indx], target.iloc[train_indx]
        X_val, y_val = synthetic_train_df.iloc[val_indx], target.iloc[val_indx]
        X_test = test_df_c.copy()

        # concat prev dataset
        X_train = pd.concat([X_train, real_world_df], axis=0)
        y_train = pd.concat([y_train, real_world_target])

        X_train = X_train.copy()
        X_val = X_val.copy()


        model = XGBClassifier(
            random_state=42, learning_rate=1e-2,
            n_estimators=3000,
            tree_method='gpu_hist',
            callbacks=[
                EarlyStopping(100, save_best=True, maximize=False),
            ]
        )

        # Fit encoder
        X_train = encoder.fit_transform(X_train)
        X_val = encoder.transform(X_val)
        X_test = encoder.transform(X_test)
        pipeline.append(encoder)

        # filter columns by RFE columns
        X_train = X_train[X_train.columns.intersection(rfe_features)]
        X_val = X_val[X_val.columns.intersection(rfe_features)]
        X_test = X_test[X_test.columns.intersection(rfe_features)]

        print('_'*50)

        model.fit(
            X_train, y_train,
            eval_set=[(X_val,y_val)],
            verbose=1000,
        )

        oof_preds.iloc[val_indx] = model.predict_proba(X_val)[:, 1]
        test_preds += model.predict_proba(X_test)[:, 1]

        train_auc.append(roc_auc_score(y_train, model.predict_proba(X_train)[:, 1]))
        val_auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))

        pipeline.append(model)
        pipelines.append(pipeline)

        print(f'Val AUC: {val_auc[-1]}')

    print()
    print(f'Mean Val AUC: {np.mean(val_auc)}')
    print(f'OOF AUC: {roc_auc_score(target, oof_preds)}')
    return test_preds, pipelines

In [20]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Generate feature engineering
synthetic_train_df = generate_features(train_df.copy())
test_df_c = generate_features(test_df.copy())
real_world_df = generate_features(real_world_df.copy())

real_world_target = real_world_df["stroke"]

# filter features 
synthetic_train_df = synthetic_train_df[features]
test_df_c = test_df_c[features]
real_world_df = real_world_df[features]

#IMPUTE VS VS
# fill nan values on original dataset
fills = real_world_df.mean(axis=0, numeric_only=True)
real_world_df.fillna(fills, inplace=True)



In [21]:
preds_ss, pipelines = XGBoost_model(synthetic_train_df, real_world_df, train_df, test_df_c, real_world_target, encoder=StringSimilarityEncoder(variables=categorical))
preds_ss /= len(pipelines)

__________________________________________________
[0]	validation_0-logloss:0.68453
[930]	validation_0-logloss:0.12712
Val AUC: 0.8864452229574844
__________________________________________________
[0]	validation_0-logloss:0.68455
[644]	validation_0-logloss:0.12763
Val AUC: 0.8772976947363869
__________________________________________________
[0]	validation_0-logloss:0.68448
[583]	validation_0-logloss:0.12509
Val AUC: 0.8803521216768916
__________________________________________________
[0]	validation_0-logloss:0.68457
[776]	validation_0-logloss:0.12507
Val AUC: 0.8954925017041582
__________________________________________________
[0]	validation_0-logloss:0.68453
[757]	validation_0-logloss:0.12277
Val AUC: 0.8922322848703216
__________________________________________________
[0]	validation_0-logloss:0.68454
[651]	validation_0-logloss:0.12744
Val AUC: 0.8863569967864446
__________________________________________________
[0]	validation_0-logloss:0.68453
[704]	validation_0-logloss:0.12507

In [22]:
preds_oh, pipelines = XGBoost_model(synthetic_train_df, real_world_df, train_df, test_df_c, real_world_target, encoder=OneHotEncoder(variables=categorical))
preds_oh /= len(pipelines)

__________________________________________________
[0]	validation_0-logloss:0.68455
[804]	validation_0-logloss:0.12690
Val AUC: 0.8832230439859867
__________________________________________________
[0]	validation_0-logloss:0.68454
[877]	validation_0-logloss:0.12842
Val AUC: 0.8747783400371956
__________________________________________________
[0]	validation_0-logloss:0.68449
[575]	validation_0-logloss:0.12492
Val AUC: 0.8806396983640081
__________________________________________________
[0]	validation_0-logloss:0.68457
[710]	validation_0-logloss:0.12429
Val AUC: 0.8992309986366734
__________________________________________________
[0]	validation_0-logloss:0.68454
[756]	validation_0-logloss:0.12264
Val AUC: 0.8949589379037233
__________________________________________________
[0]	validation_0-logloss:0.68455
[644]	validation_0-logloss:0.12700
Val AUC: 0.8885967474924531
__________________________________________________
[0]	validation_0-logloss:0.68453
[706]	validation_0-logloss:0.12474

In [23]:
preds_ss

id
15304    0.051600
15305    0.172296
15306    0.001811
15307    0.051274
15308    0.011238
           ...   
25503    0.001839
25504    0.009145
25505    0.001947
25506    0.003292
25507    0.001868
Name: stroke, Length: 10204, dtype: float64

In [24]:
preds_oh

id
15304    0.049530
15305    0.159378
15306    0.001665
15307    0.042521
15308    0.007461
           ...   
25503    0.001647
25504    0.009171
25505    0.001783
25506    0.003157
25507    0.001709
Name: stroke, Length: 10204, dtype: float64

As we saw in our LogisticRegressor, StringSimilarityEncoder fits *very slightly* better than OneHotEncoder.

In [25]:
preds_ss.rename('stroke', inplace=True)
preds_ss.to_csv('submission_ss.csv') # Public Score: 0.8672
preds_oh.rename('stroke', inplace=True)
preds_oh.to_csv('submission_oh.csv')  # Public Score: 0.8670